In [1]:
from adsabs_utils import combine_csv_data, aff_country, group_eu_countries, extract_primary_journal, add_country_code_column, SJRMatcher, classify_grouped_keywords
#from ads_data_aff import aff_country

import pandas as pd
import ast
import re
from collections import Counter

In [3]:
all_data =combine_csv_data()

Dataframe with keyword: AGN - 18651 articles
Dataframe with keyword: ArtificialIntelligence - 30982 articles
Dataframe with keyword: Asteroseismology - 2809 articles
Dataframe with keyword: Astrobiology - 356469 articles
Dataframe with keyword: Cosmology - 146547 articles
Dataframe with keyword: DeepLearning - 226480 articles
Dataframe with keyword: Exoplanet - 12571 articles
Dataframe with keyword: Galaxy - 154130 articles
Dataframe with keyword: MachineLearning - 220535 articles
Dataframe with keyword: Star - 185370 articles
1354544 retrived articles
931105 unique articles
2486 articles without author
928619 unique articles with author
Saved in ads_data/all_data_unique.parquet


1005825 retrived articles
590709 unique articles
806 articles without author
589903 unique articles with author


In [4]:
all_data.columns

Index(['bibcode', 'my_keyword', 'abstract', 'aff', 'arxiv_class', 'author',
       'bibstem', 'database', 'date', 'doctype', 'doi', 'first_author',
       'keyword', 'pub', 'pubdate', 'title', 'year', 'read_count', 'property',
       'citation_count', 'references'],
      dtype='object')

In [5]:
aff_country(input_file ='ads_data/all_data_unique.parquet', output_file='ads_data/all_data_aff.parquet')

Calif not found in regex
Roc not found in ISO3
 Hamburg not found in regex
Mxico not found in regex
Mxico not found in regex
Brasil not found in regex
Sezione Di Napoli not found in regex
Roc not found in ISO3
Mxico not found in regex
Danmarks Grundforskningsfond not found in regex
Mxico not found in regex
Umr  Httpwwwompobsmipfromp not found in regex
 Garching Bei Mnchen not found in regex
Yugoslavia not found in regex
Mxico not found in regex
Texas  not found in regex
Ohio  not found in regex
Maryland  not found in regex
Nasasupported Undergraduate Research Associate not found in regex
Crimean Branch not found in regex
Texas  not found in regex
Mxico not found in regex
Under Cooperative Agreement With The National Science Foundation not found in regex
National Optical Astronomy Observatories not found in regex
National Observatory Of Athens not found in regex
Operated By The Instituto De Astrofsica De Canarias not found in regex
Arizona  not found in regex
Noao Wiyn Queue Investigato

Saved data to ads_data/all_data_aff.parquet
Done


In [6]:
ads_data_aff = pd.read_parquet('ads_data/all_data_aff.parquet')

In [7]:
#Extracts the primary journal name from a stringified list in the `bibstem` column.
ads_data_aff = extract_primary_journal(ads_data_aff, bibstem_col='bibstem', output_col='journal')
#Groups EU countries under 'European Union' in the given DataFrame column.
ads_data_aff = group_eu_countries(ads_data_aff, country_col='first_author_aff_country_final_valid', output_col='country_grouped')
#Adds a new column with 2-letter ISO country codes based on a country name column.
ads_data_aff = add_country_code_column(ads_data_aff)

# Save the DataFrame to a Parquet file.
ads_data_aff.to_parquet('ads_data/all_data_aff.parquet', index=False)

[!] Could not find ISO code for: Cote d'Ivoire
[!] Could not find ISO code for: Cote d'Ivoire
[!] Could not find ISO code for: Cote d'Ivoire
[!] Could not find ISO code for: Falkland Islands
[!] Could not find ISO code for: Falkland Islands
[!] Could not find ISO code for: Vatican
[!] Could not find ISO code for: Congo Republic
[!] Could not find ISO code for: Vatican
[!] Could not find ISO code for: Saint-Martin
[!] Could not find ISO code for: Palestine
[!] Could not find ISO code for: Palestine
[!] Could not find ISO code for: Vatican
[!] Could not find ISO code for: Vatican
[!] Could not find ISO code for: Palestine
[!] Could not find ISO code for: Saint-Martin
[!] Could not find ISO code for: Falkland Islands
[!] Could not find ISO code for: Falkland Islands
[!] Could not find ISO code for: Palestine
[!] Could not find ISO code for: Saint-Martin
[!] Could not find ISO code for: Congo Republic
[!] Could not find ISO code for: Reunion
[!] Could not find ISO code for: Palestine
[!] C

In [8]:
#exoplanets = all_data[all_data['my_keyword'].str.contains('exoplanet', case=False, na=False)]
#exoplanets.shape

In [ ]:
matcher = SJRMatcher(
    sjr_path="different_data/scimagojr_combined_2001_2023.csv",
    ads_path="ads_data/all_data_aff.parquet"
)
sjr_df_updated_final = matcher.run(cutoff=0.9, output_path="different_data/cimagojr_combined_fuzzy_extended_final.csv")


ads_data_aff_updated = ads_data_aff.merge(
    sjr_df_updated_final,
    left_on='pub',
    right_on='Title',
    how='left'
)
ads_data_aff_updated.to_parquet('ads_data/all_data_aff_sjr.parquet', index=False)

In [ ]:
# Testing the impact of updating the SJR data

ads_data_aff_old = ads_data_aff.merge(
    sjr_df_updated_final,
    left_on='pub',
    right_on='matched_sjr_title_full',
    how='left'
)


In [ ]:
ads_data_aff_updated[ads_data_aff_updated.Title == 'The Astrophysical Journal']['similarity'].head(1)

1951    0.913043
Name: similarity, dtype: float64

In [ ]:
ads_data_aff_updated[ads_data_aff_updated['SJR Best Quartile'] != ads_data_aff_updated['SJR Best Quartile']].shape

(43525, 38)

In [ ]:
ads_data_aff_updated[ads_data_aff_updated['SJR Best Quartile'] != ads_data_aff_updated['SJR Best Quartile']]['pub']

0         American Institute of Physics Conference Series
1         American Institute of Physics Conference Series
1418                Acoustical Society of America Journal
1419                Acoustical Society of America Journal
1420                Acoustical Society of America Journal
                               ...                       
928614                        npj Sustainable Agriculture
928615                        npj Sustainable Agriculture
928616                        npj Sustainable Agriculture
928617                         npj Scientific of Learning
928618                           npj Urban Sustainability
Name: pub, Length: 43525, dtype: object

In [ ]:
ads_data_aff_old[ads_data_aff_old['SJR Best Quartile'] != ads_data_aff_old['SJR Best Quartile']].shape

(178194, 38)

### Changing keywords of AI and Grouping my keywords: AI, Astronomy, Astrobiology, Cosmology

In [ ]:
all_data_aff_sjr = pd.read_parquet('ads_data/all_data_aff_sjr.parquet')

# Define mapping of terms to unify
replace_map = {
    "MachineLearning": "AI",
    "ArtificialIntelligence": "AI",
    "DeepLearning": "AI"
}

def consolidate_keywords(row, mapping):
    keywords = row.split(',')
    new_keywords = []
    for kw in keywords:
        kw = kw.strip()
        if kw in mapping:
            new_keywords.append(mapping[kw])
        else:
            new_keywords.append(kw)
    return ', '.join(sorted(set(new_keywords)))  # Remove duplicates & sort

# Apply to your DataFrame
all_data_aff_sjr['my_keyword'] = all_data_aff_sjr['my_keyword'].fillna("").apply(lambda row: consolidate_keywords(row, replace_map))

# Grouping my keywords: AI, Astronomy, Astrobiology, Cosmology
all_data_aff_sjr['grouped_keywords'] = all_data_aff_sjr['my_keyword'].apply(classify_grouped_keywords)
all_data_aff_sjr.to_parquet('ads_data/all_data_aff_sjr.parquet', index=False)

### Clean DataFrame

In [ ]:
all_data_aff_sjr.head(1)

,bibcode,my_keyword,abstract,aff,arxiv_class,author,bibstem,database,date,doctype,...,journal,country_grouped,country_code,Title,matched_sjr_title_full,similarity,source,SJR Best Quartile,Last_seen_year,grouped_keywords
0,1977ElL....13..717B,Astrobiology,Many coastal communities in Indonesia are depe...,"['Ticra A/S, Copenhagen, Denmark', 'Standard E...",None,"['Balling, P.', 'Jacobsen, J.']","['ElL', 'ElL....13']",['physics'],2024-02-01T00:00:00Z,article,...,ElL,European Union,dk,None,None,NaN,None,None,NaN,Astrobiology


In [ ]:
all_data_aff_sjr = pd.read_parquet('ads_data/all_data_aff_sjr.parquet')
ads_data_country_clean1 = all_data_aff_sjr[['bibcode','my_keyword', 'grouped_keywords', 'pub', 'year', 
       'citation_count', 'first_author_aff_country_final_valid',
       'journal', 'country_grouped', 'country_code',  'SJR Best Quartile']].copy()
ads_data_country_clean2 = ads_data_country_clean1[ads_data_country_clean1['first_author_aff_country_final_valid'].notna()]
ads_data_country_clean2.rename(columns={'SJR Best Quartile': 'sjr_quartile'}, inplace=True)
ads_data_country_clean2.to_parquet('ads_data/all_data_country_clean.parquet', index=False)
ads_data_country_clean2.to_parquet('ads_stats_github/all_data_country_clean.parquet', index=False)

C:\Users\adbva\AppData\Local\Temp\ipykernel_37088\3209654499.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ads_data_country_clean2.rename(columns={'SJR Best Quartile': 'sjr_quartile'}, inplace=True)


In [ ]:
ads_data_country_clean2.shape

(746649, 11)

In [ ]:
ads_data_country_clean2.to_parquet('ads_data/all_data_country_clean.parquet', index=False)